In [1]:
import numpy as np
import cv2 as cv

In [2]:
FLOYD = 1/16 * np.array([
    [0, 0, 7],
    [3, 5, 1]
])


In [3]:
def stipple(img, error, zigzag):
    # error distribution size
    eX = error.shape[1] // 2
    eY = error.shape[0] - 1

    # add border and change array type to float
    # f(x,y) image
    img = cv.copyMakeBorder(img,
                            top=0, bottom=eY,
                            left=eX, right=eX,
                            borderType=cv.BORDER_CONSTANT
                            )
    img = img.astype(float, copy=False)

    # size for loop
    dX = img.shape[1] - eX
    dY = img.shape[0] - eY

    # g(x,y) image
    out = np.zeros(img.shape)

    # reverse error
    error_rev = np.fliplr(error)

    for y in range(dY):
        # reverse if is odd
        reverse = (y % 2 == 1) if zigzag else False
        error_curr = error_rev if reverse else error
        
        for x in range(eX, dX)[::-1 if reverse else 1]:
            out[y, x] = 0 if img[y, x] < 128 else 255

            diff = img[y, x] - out[y, x]
            slice = img[y:y+eY+1, x-eX:x+eX+1]
            slice += (error_curr * diff)

    out = out[:dY, eX:dX]
    return out


In [4]:
inp_img = cv.imread('./images/baboon.png', cv.IMREAD_COLOR)

b = set(inp_img[:,:,0].flatten())
g = set(inp_img[:,:,1].flatten())
r = set(inp_img[:,:,2].flatten())

inp_img = cv.cvtColor(inp_img, cv.COLOR_BGR2HSV)

h = set(inp_img[:,:,0].flatten())
s = set(inp_img[:,:,1].flatten())
v = set(inp_img[:,:,2].flatten())

print('h: ', len(h))
print('s: ', len(s))
print('v: ', len(v))

#out_img = np.zeros(inp_img.shape)
out_img = np.copy(inp_img)

out_img[:,:,2] = stipple(inp_img[:,:,2], FLOYD, True)

out_img = cv.cvtColor(out_img, cv.COLOR_HSV2BGR)

cv.imwrite('./images/baboon_test_floyd1_color.png', out_img)

h:  180
s:  256
v:  256


True